# Chapter 7 - Advanced Agent Systems: Multi-Agent Collaboration and Observability

## Prerequisites and Setup

Please refer to the [Strands Quickstart Guide](https://strandsagents.com/latest/documentation/docs/user-guide/quickstart/) for  details on:
- **Installation**: Setting up the Strands SDK and dependencies
- **Credentials**: Configuring AWS credentials for Amazon Bedrock
- **Basic Concepts**: Understanding agents, tools, and model providers
- **Project Setup**: Creating your first Strands agent

The quickstart guide provides essential setup instructions and foundational concepts that will help you understand the advanced patterns demonstrated in this notebook.

## Overview
This notebook demonstrates advanced Strands capabilities including multi-agent systems and observability. We'll build a collaborative customer service system where specialized agents work together to handle complex customer requests.

## What You'll Learn
- **Multi-Agent Systems**: How agents can collaborate and delegate tasks
- **Observability**: Monitoring agent performance and tool usage with OpenTelemetry
- **Advanced Workflows**: Complex business processes with multiple agents
- **Error Handling**: Robust error recovery and fallback mechanisms
- **Production Patterns**: Best practices for building scalable agent systems

## System Architecture
Our multi-agent system includes:
- **Order Agent**: Handles order-related queries and operations
- **Inventory Agent**: Manages product availability and stock levels  
- **Orchestrator Agent**: Coordinates between agents and routes requests
- **Observability**: Tracks agent performance and system health with OpenTelemetry


## 1. Setup and Dependencies


In [ ]:
# Install required packages
%pip install strands-agents strands-agents-tools boto3 opentelemetry-api opentelemetry-sdk


In [ ]:
import json
import logging
import time
from datetime import datetime
from typing import Dict, List, Optional, Any
import uuid

# Strands imports
from strands import Agent, tool
from strands_tools import calculator
from strands.models import BedrockModel

# Observability imports
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.sdk.resources import Resource

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configure OpenTelemetry for observability
trace.set_tracer_provider(TracerProvider(resource=Resource.create({"service.name": "multi-agent-system"})))
tracer = trace.get_tracer(__name__)

print("Dependencies loaded successfully!")


## 2. Agents as Tools Pattern

This example demonstrates the "Agents as Tools" pattern where specialized agents are wrapped as callable functions that can be used by other agents, creating a hierarchical structure with clear separation of concerns.


In [ ]:
# Multi-Agent System Overview
# This example demonstrates how specialized agents can work together
# Each agent has specific expertise and tools, creating a collaborative system

print("Multi-Agent System Overview:")
print("- Order Management Agent: Handles order queries and operations")
print("- Inventory Management Agent: Manages product availability and stock")
print("- Orchestrator Agent: Routes requests to appropriate specialists")
print("- Observability: Tracks performance with OpenTelemetry")
print("- Agents as Tools: Specialists can be used by other agents")


## 3. Shared Data and Tools


In [ ]:
# Sample data stores
orders_db = {
    "ORD001": {"customer_id": "CUST001", "items": ["macbook-pro-13", "wireless-mouse"], "status": "processing", "total": 1299.99},
    "ORD002": {"customer_id": "CUST002", "items": ["noise-cancelling-headphones"], "status": "shipped", "total": 199.99},
    "ORD003": {"customer_id": "CUST001", "items": ["mechanical-keyboard"], "status": "delivered", "total": 89.99}
}

inventory_db = {
    "macbook-pro-13": {
        "stock": 8, 
        "price": 1199.99, 
        "category": "laptops",
        "brand": "Apple",
        "model": "MacBook Pro 13-inch",
        "specs": "M2 chip, 8GB RAM, 256GB SSD, 13.3-inch Retina display",
        "description": "Powerful and portable laptop perfect for professionals and students"
    },
    "macbook-air-15": {
        "stock": 12, 
        "price": 1299.99, 
        "category": "laptops",
        "brand": "Apple",
        "model": "MacBook Air 15-inch",
        "specs": "M2 chip, 8GB RAM, 256GB SSD, 15.3-inch Liquid Retina display",
        "description": "Ultra-thin laptop with stunning 15-inch display and all-day battery life"
    },
    "dell-xps-13": {
        "stock": 15, 
        "price": 1099.99, 
        "category": "laptops",
        "brand": "Dell",
        "model": "XPS 13",
        "specs": "Intel i7, 16GB RAM, 512GB SSD, 13.4-inch 4K display",
        "description": "Premium Windows laptop with exceptional build quality and performance"
    },
    "wireless-mouse": {
        "stock": 50, 
        "price": 29.99, 
        "category": "accessories",
        "brand": "Logitech",
        "model": "MX Master 3",
        "specs": "Wireless, ergonomic design, 70-day battery life",
        "description": "Premium wireless mouse with precision tracking and comfortable grip"
    },
    "noise-cancelling-headphones": {
        "stock": 25, 
        "price": 199.99, 
        "category": "audio",
        "brand": "Sony",
        "model": "WH-1000XM5",
        "specs": "Active noise cancellation, 30-hour battery, wireless",
        "description": "Industry-leading noise-cancelling headphones with exceptional sound quality"
    },
    "mechanical-keyboard": {
        "stock": 30, 
        "price": 89.99, 
        "category": "accessories",
        "brand": "Keychron",
        "model": "K2 V2",
        "specs": "Bluetooth/wired, RGB backlight, Gateron switches",
        "description": "Compact mechanical keyboard with wireless connectivity and customizable lighting"
    }
}



In [ ]:
# Shared tools for all agents
@tool
def log_agent_action(agent_name: str, action: str, details: str) -> str:
    """Log agent actions for observability"""
    with tracer.start_as_current_span("log_agent_action") as span:
        span.set_attribute("agent_name", agent_name)
        span.set_attribute("action", action)
        logger.info(f"Agent {agent_name} performed {action}: {details}")
        return f"Logged action: {agent_name} - {action}"

print("Shared data and tools initialized!")


## 4. Specialized Agents as Tools


In [ ]:
# Configure Bedrock model
bedrock_model = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.3,
    streaming=True
)

# 1. First, create the order management tools
@tool
def check_order_status(order_id: str) -> str:
    """Check the status of an order"""
    with tracer.start_as_current_span("check_order_status") as span:
        span.set_attribute("order_id", order_id)
        order = orders_db.get(order_id.upper())
        if order:
            return f"Order {order_id}: Status={order['status']}, Items={order['items']}, Total=${order['total']}"
        return f"Order {order_id} not found"

@tool
def create_order(customer_id: str, items: List[str]) -> str:
    """Create a new order"""
    with tracer.start_as_current_span("create_order") as span:
        span.set_attribute("customer_id", customer_id)
        order_id = f"ORD{len(orders_db) + 1:03d}"
        total = sum(inventory_db.get(item, {}).get('price', 0) for item in items)
        orders_db[order_id] = {
            "customer_id": customer_id,
            "items": items,
            "status": "processing",
            "total": total
        }
        return f"Created order {order_id} for customer {customer_id} with items {items}, total: ${total}"

# 2. Create the order management agent and add tools
order_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are a specialized Order Management Agent. You handle:
    - Checking order status and details
    - Creating new orders
    - Providing order history and tracking information
    - Explaining order processes and policies
    
    Always provide clear, helpful responses about orders and order management.""",
    tools=[check_order_status, create_order, log_agent_action]
)

print("Order Management Agent created successfully!")


In [ ]:
# 3. Create the inventory management tools
@tool
def check_product_availability(product_name: str) -> str:
    """Check if a product is available and get detailed information"""
    with tracer.start_as_current_span("check_product_availability") as span:
        span.set_attribute("product_name", product_name)
        
        # Try exact match first
        product = inventory_db.get(product_name.lower())
        
        # If no exact match, try partial matching for specific products
        if not product:
            search_term = product_name.lower()
            if "macbook pro" in search_term:
                product = inventory_db.get("macbook-pro-13")
            elif "macbook air" in search_term:
                product = inventory_db.get("macbook-air-15")
            elif "dell xps" in search_term or "xps" in search_term:
                product = inventory_db.get("dell-xps-13")
            elif "mouse" in search_term:
                product = inventory_db.get("wireless-mouse")
            elif "headphone" in search_term or "headset" in search_term:
                product = inventory_db.get("noise-cancelling-headphones")
            elif "keyboard" in search_term:
                product = inventory_db.get("mechanical-keyboard")
            elif "laptop" in search_term:
                # Return all available laptops
                laptops = {k: v for k, v in inventory_db.items() if v.get("category") == "laptops"}
                if laptops:
                    result = "Available Laptops:\n"
                    for key, laptop in laptops.items():
                        result += f"\n• {laptop['brand']} {laptop['model']}\n"
                        result += f"  Price: ${laptop['price']}\n"
                        result += f"  Stock: {laptop['stock']} units\n"
                        result += f"  Specs: {laptop['specs']}\n"
                        result += f"  Description: {laptop['description']}\n"
                    return result
        
        if product:
            return f"""Product Details:
• Brand: {product['brand']}
• Model: {product['model']}
• Price: ${product['price']}
• Stock: {product['stock']} units available
• Category: {product['category']}
• Specifications: {product['specs']}
• Description: {product['description']}"""
        
        return f"Product '{product_name}' not found in inventory. Available categories: laptops, accessories, audio"

@tool
def reserve_inventory(product_name: str, quantity: int) -> str:
    """Reserve inventory for an order"""
    with tracer.start_as_current_span("reserve_inventory") as span:
        span.set_attribute("product_name", product_name)
        span.set_attribute("quantity", quantity)
        product = inventory_db.get(product_name.lower())
        if product and product['stock'] >= quantity:
            product['stock'] -= quantity
            return f"Reserved {quantity} units of {product_name}. Remaining stock: {product['stock']}"
        return f"Cannot reserve {quantity} units of {product_name}. Available stock: {product.get('stock', 0)}"

# 4. Create the inventory management agent and add tools
inventory_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are a specialized Inventory Management Agent. You handle:
    - Checking product availability and stock levels
    - Providing product details and specifications
    - Managing inventory reservations
    - Explaining product categories and pricing
    
    Always provide accurate, up-to-date information about products and inventory.""",
    tools=[check_product_availability, reserve_inventory, log_agent_action]
)

print("Inventory Management Agent created successfully!")


In [ ]:
# 5. Use specialized agents as tools for the orchestrator
@tool
def order_management_agent(query: str) -> str:
    """Handle order-related queries including status checks, order creation, and order history"""
    with tracer.start_as_current_span("order_management_agent") as span:
        span.set_attribute("query", query)
        response = order_agent(query)
        return str(response)

@tool
def inventory_management_agent(query: str) -> str:
    """Handle inventory and product-related queries including availability checks, stock levels, and reservations"""
    with tracer.start_as_current_span("inventory_management_agent") as span:
        span.set_attribute("query", query)
        response = inventory_agent(query)
        return str(response)

# 6. Create orchestrator agent with agent tools
orchestrator_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are a Customer Support Orchestrator that routes queries to specialized agents:

- For order-related queries (status, creation, history) → Use the order_management_agent tool
- For inventory and product queries (availability, stock, details) → Use the inventory_management_agent tool  
- For simple questions not requiring specialized knowledge → Answer directly
- For complex queries requiring multiple domains → Use multiple specialized agents as needed

Always select the most appropriate tool(s) based on the user's query. Provide clear, helpful responses by leveraging the expertise of your specialized agents.""",
    tools=[order_management_agent, inventory_management_agent, log_agent_action, calculator]
)

print("Orchestrator Agent created successfully!")


## 5. Testing the Multi-Agent System


In [ ]:
# Test the Multi-Agent system
def process_customer_request(customer_id: str, message: str) -> str:
    """Process customer request using the orchestrator agent"""
    with tracer.start_as_current_span("process_customer_request") as span:
        span.set_attribute("customer_id", customer_id)
        span.set_attribute("message", message)
        
        # Process with orchestrator agent (which uses specialized agents as tools)
        logger.info(f"Processing request for customer {customer_id}: {message}")
        response = orchestrator_agent(message)
        
        return response

print("Agents as Tools system ready!")


In [ ]:
# Test 1: Order-related query
print("=== Test 1: Order Status Query ===")
response1 = process_customer_request("CUST001", "Can you check the status of my order ORD001?")
print(f"Response: {response1}")
print("\n" + "="*60 + "\n")


In [ ]:
# Test 2: Inventory-related query
print("=== Test 2: Product Availability Query ===")
response2 = process_customer_request("CUST002", "Do you have MacBook Pro in stock? What's the price and specifications?")
print(f"Response: {response2}")
print("\n" + "="*60 + "\n")


In [ ]:
# Test 3: Complex multi-step request
print("=== Test 3: Complex Multi-Step Request ===")
response3 = process_customer_request("CUST001", "I want to order a MacBook Pro and a wireless mouse. Can you check if they're available and create an order for me?")
print(f"Response: {response3}")
print("\n" + "="*60 + "\n")


## 6. Observability and System Metrics


In [ ]:
# Display system metrics
print("=== System Metrics ===")
print(f"Total Orders: {len(orders_db)}")
print(f"Total Products: {len(inventory_db)}")
print(f"Timestamp: {datetime.now().isoformat()}")
print(f"Architecture: Multi-Agent System with Agents as Tools pattern")

print("\n=== Observability Features ===")
print("OpenTelemetry Integration:")
print("- Distributed tracing across all agents")
print("- Performance monitoring and metrics")
print("- Structured logging with correlation IDs")
print("- Agent action tracking and analysis")
print("- Production-ready observability stack")


## 7. Key Benefits of Multi-Agent Systems

### Advantages of the Agents as Tools Pattern:

1. **Separation of Concerns**: Each agent has a focused area of responsibility, making the system easier to understand and maintain

2. **Hierarchical Delegation**: The orchestrator decides which specialist to invoke, creating a clear chain of command

3. **Modular Architecture**: Specialists can be added, removed, or modified independently without affecting the entire system

4. **Improved Performance**: Each agent can have tailored system prompts and tools optimized for its specific task

5. **Clear Tool Documentation**: Descriptive docstrings explain each agent's expertise and capabilities

6. **Focused System Prompts**: Each specialized agent is tightly focused on its domain

### Key Benefits of Observability Integration:

1. **Distributed Tracing**: Track requests across multiple agents and tools

2. **Performance Monitoring**: Identify bottlenecks and optimize agent performance

3. **Error Tracking**: Quickly identify and debug issues in complex multi-agent workflows

4. **Usage Analytics**: Understand which agents and tools are most frequently used

5. **Production Readiness**: Enterprise-grade monitoring for scalable deployments

6. **Debugging Support**: Rich context for troubleshooting agent interactions

### Real-World Use Cases:

- **Customer Support Automation**: Monitor response times and success rates across different support scenarios
- **E-Commerce Assistants**: Track conversion rates and customer satisfaction across agent interactions
- **Help Desk Systems**: Analyze ticket resolution patterns and agent performance
- **Enterprise Workflows**: Monitor complex business processes involving multiple AI agents

### Production Benefits:

- **Scalability**: Monitor system performance as you scale to thousands of concurrent users
- **Reliability**: Detect and respond to issues before they impact customers
- **Optimization**: Use metrics to continuously improve agent performance and accuracy
- **Compliance**: Maintain audit trails and compliance reporting for regulated industries


## 8. Scaling to Production with Amazon Bedrock AgentCore

### From Prototype to Production

While the multi-agent system we've built demonstrates powerful capabilities, taking AI agents from proof-of-concept to production requires additional considerations around scalability, security, monitoring, and operational excellence.

**Amazon Bedrock AgentCore** is a comprehensive platform that accelerates AI agents into production with the scale, reliability, and security critical to real-world deployment. It's both framework-agnostic and model-agnostic, giving you the flexibility to deploy and operate advanced AI agents securely and at scale.

### Key Production Capabilities

**Amazon Bedrock AgentCore** provides several essential services for production deployment:

- **Runtime**: A secure, serverless runtime that scales your agents automatically, regardless of framework or model choice
- **Gateway**: Automatically converts APIs, Lambda functions, and existing services into MCP-compatible tools
- **Memory**: Fully-managed memory infrastructure for building rich, personalized agent experiences
- **Identity**: Seamless agent identity and access management across AWS services and third-party applications
- **Observability**: Unified operational dashboards with OpenTelemetry support for tracing, debugging, and monitoring
- **Built-in Tools**: Enterprise-grade Code Interpreter and Browser tools for enhanced agent capabilities

### Framework Agnostic

Whether you're building with **Strands Agents**, CrewAI, LangGraph, LlamaIndex, or any other framework—and running them on any Large Language Model—Amazon Bedrock AgentCore provides the infrastructure to support them without requiring code rewrites.

### Getting Started

To learn more about deploying your agents to production:

- **🚀 Quick Start**: Get your first agent running in 5 minutes with the [AgentCore Runtime Quickstart Guide](https://aws.github.io/bedrock-agentcore-starter-toolkit/user-guide/runtime/quickstart.html)
- **📚 Samples & Tutorials**: Explore the [Amazon Bedrock AgentCore Samples repository](https://github.com/awslabs/amazon-bedrock-agentcore-samples) for comprehensive examples and tutorials
- **🌐 Official Documentation**: Visit the [Amazon Bedrock AgentCore homepage](https://aws.amazon.com/bedrock/agentcore/) for detailed information about capabilities and pricing

### Next Steps

1. **Explore the Samples**: Review the AgentCore samples to understand production patterns
2. **Plan Your Architecture**: Consider how AgentCore services can enhance your current agent system
3. **Start with Runtime**: Begin by deploying your existing Strands agents to AgentCore Runtime
4. **Add Production Features**: Gradually integrate Gateway, Memory, and Observability services

Amazon Bedrock AgentCore eliminates the undifferentiated heavy lifting of building and managing specialized agent infrastructure, letting you focus on your business logic while ensuring your agents can scale securely and reliably in production environments.
